# DoIT Schedule Building
### Ray Smith - ECE 524

In [1]:
ENV["GUROBI_HOME"] = "/Library/gurobi911/mac64"
ENV["GRB_LICENSE_FILE"] = "/Library/gurobi911/gurobi.lic"
import Pkg

In [2]:
# Imports
using JuMP
Pkg.add("Gurobi")
Pkg.build("Gurobi")
using Gurobi

using CSV
using JSON
using DataFrames
using OrderedCollections

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [

    Fetching: [===================>                     

    Fetching:

    Fetching: [======================================>

Fetching: [========================================>]  99.9 %

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
   Building Gurobi → `~/.julia/packages/Gurobi/JcjAE/deps/build.log`
┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1278


In [134]:
# Schedule Rules

# General Schedule Layout
hours = 1:32
hours_real = 7:0.5:22.5
days = 1:7

# Shift Types and Training Requirements
shifts = ["HDQA", "Phones", "Chat/Email", "Walk-In"]
training_levels = ["Pick 1", "Walk-In", "Pick 3", "Chat/Email", "HDQA", "SLP"]
requirements = Dict()
requirements[shifts[1]] = [training_levels[5]]
requirements[shifts[2]] = [training_levels[1]]
requirements[shifts[3]] = [training_levels[4]]
requirements[shifts[4]] = [training_levels[2]]

# Allowed Flexibility in (Extra) Agents Scheduled
flexibility = Dict()
flexibility[shifts[1]] = 0
flexibility[shifts[2]] = 1
flexibility[shifts[3]] = 1
flexibility[shifts[4]] = 1

# Other Rules
adv_phones_req = 1 # min number of pick 3 trained agents scheduled over a phones shift
min_shift_length = 2 # in 1/2 hour blocks, must be even
agent_min = 15
agent_max = 25

25

In [121]:
# Data Input

# Schedule Shifts
raw_week = CSV.read(joinpath(@__DIR__, "week_req.csv"), DataFrame)
week_shift_types = names(raw_week)
week = Matrix(raw_week)
raw_weekend = CSV.read(joinpath(@__DIR__, "weekend_req.csv"), DataFrame)
weekend_shift_types = names(raw_weekend)
weekend = Matrix(raw_weekend)

weekday = [OrderedDict() for i in hours]
weekendday = [OrderedDict() for i in hours]
for i in hours
    j = 1
    for shift in week_shift_types
        weekday[i][shift] = week[i, j]
        j += 1
    end
    k = 1
    for shift in weekend_shift_types
        weekendday[i][shift] = weekend[i, k]
        k += 1
    end
end

# Availability
string_data = join(readlines("availability.json"))
raw_avail = JSON.parse(string_data)
agents = keys(raw_avail)
avail = OrderedDict()
for agent in agents
    raw_agent_avail = raw_avail[agent]
    agent_avail = OrderedDict()
    for i in hours
        agent_avail[i] = raw_agent_avail[i]
    end
    avail[agent] = agent_avail
end

# Training
string_data = join(readlines("agent_training.json"))
raw_trainings = JSON.parse(string_data)
all_agents = keys(raw_trainings)
trainings = OrderedDict()
for agent in all_agents
    raw_agent_trainings = raw_trainings[agent]
    base_job = raw_agent_trainings[:"Base"]
    agent_trainings = raw_agent_trainings[:"Trainings"]
    if startswith(base_job, "SLP")
        append!(agent_trainings, ["SLP"])
    end
    trainings[agent] = agent_trainings
end

In [122]:
# Model Helper Functions

# Check if agent has training
function has_training(agent, training_level)
    return training_level in trainings[agent]
end

# Check if agent meets all training requirements for a shift
function meets_requirements(agent, shift)
    shift_req = requirements[shift]
    for training in shift_req
        if !has_training(agent, training)
            return false
        end
    end
    return true
end

# Check if agent is available (green/yellow) at a day/time
function shift_available(agent, day, time)
    agent_avail = avail[agent]
    time_slice = agent_avail[time]
    color = time_slice[day]
    
    if color == "G" || color == "Y"
        return true
    else
        return false
    end
end

# Get shift types for a day
function get_shift_types(day)
    if (day == 1 || day == 7)
        return weekend_shift_types
    else
        return week_shift_types
    end
end

# Get shift requirements for a day
function get_day_req(day)
    if (day == 1 || day == 7)
        return weekendday
    else
        return weekday
    end
end

# Get agent total available hours
function get_total_hours(agent)
    total = 0
    for day in days
        for hour in hours
            if (shift_available(agent, day, hour))
                total += 1
            end
        end
    end
    return total
end

# Get agent min hours
function get_min_hours(agent)
    return min(get_total_hours(agent), agent_min)
end

get_min_hours (generic function with 1 method)

In [141]:
# Model Setup

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, sch[agents, days, hours], Bin) # If agent scheduled
@variable(m, req[agents, shifts], Bin) # If agent meets shift requirement
@variable(m, adv[agents], Bin) # If agent is adv. phones trained

# Set shift eligibility
for agent in agents
    for shift in shifts
        @constraint(m, req[agent, shift] == meets_requirements(agent, shift))
    end
    @constraint(m, adv[agent] == has_training(agent, "Pick 3"))
end

# Shift requirement constraints
for day in days
    shift_types = get_shift_types(day)
    day_req = get_day_req(day)
    
    for hour in hours
        max_workers = 0
        for shift in shift_types
            # Ensure enough trained agents on each shift
            @constraint(m, sum(sch[agent, day, hour] * req[agent, shift] for agent in agents) >= day_req[hour][shift])
            max_workers += day_req[hour][shift] + flexibility[shift]
        end
        # Ensure no excessive scheduling of extra workers
        @constraint(m , sum(sch[agent, day, hour] for agent in agents) <= max_workers)
        
        # Advanced phone agent requirement
        @constraint(m, sum(sch[agent, day, hour] * adv[agent] for agent in agents) >= adv_phones_req)
    end
end

# Ensure agents only scheduled when green/yellow
for agent in agents
    for day in days
        for hour in hours
            @constraint(m, sch[agent, day, hour] <= shift_available(agent, day, hour))
        end
    end
end

# Ensure each agent works within the min/max allowed hour range
for agent in agents
    @constraint(m, sum(sch[agent, day, hour] for day in days, hour in hours) >= get_min_hours(agent))
    @constraint(m, sum(sch[agent, day, hour] for day in days, hour in hours) <= agent_max)
end

# Don't allow shifts of < minimum duration
half_shift = convert(UInt8, min_shift_length / 2)
for agent in agents
    for day in days
        for hour in hours[min_shift_length:end-half_shift]
            @constraint(m, sum(sch[agent, day, h] for h in hour-half_shift:hour+half_shift) >= sch[agent, day, hour] * min_shift_length)
        end
    end
end

# Schedule the minimum amount of agents possible
@objective(m, Min, sum(sch))

# Solve the scheduling problem
optimize!(m)
println("Total Hours Scheduled: ", objective_value(m))
# 7542 is max using all green availability
# 12092 is max using all green/yellow availability

Academic license - for non-commercial use only - expires 2021-05-06
Total Hours Scheduled: 1110.0


In [124]:
# Assign roles


In [144]:
# View agent schedule

X = value.(sch)
X["RAYS",:,:]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:7
    Dimension 2, 1:32
And data, a 7×32 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [148]:
# Show Daily Schedule Nicely

schedule = OrderedDict()
for day in days
    schedule[day] = OrderedDict()
    for hour in hours
        schedule[day][hours_real[hour]] = []
        for agent in agents
            if X[agent, day, hour] == 1
                append!(schedule[day][hours_real[hour]], [agent])
            end
        end
    end
end

day_to_show = 1 # Monday
for hour in hours_real
    println(hour, " : ", schedule[day_to_show][hour])
end

7.0 : Any["AWGR", "TBOS", "DUNN"]
7.5 : Any["JOBA", "KELL", "DUNN"]
8.0 : Any["JOBA", "KELL", "GOFF"]
8.5 : Any["LEBO", "GOFF", "AARK", "JEMA"]
9.0 : Any["KALA", "LEBO", "AARK", "JEMA"]
9.5 : Any["KALA", "LEBO", "AARK"]
10.0 : Any["KALA", "LKRE", "AARK", "GABE"]
10.5 : Any["AWGR", "LKRE", "JLHE", "GABE"]
11.0 : Any["AWGR", "LKRE", "JLHE", "GABE"]
11.5 : Any["LKRE", "JLHE", "GENI", "GABE"]
12.0 : Any["HUNT", "LKRE", "GENI", "GABE"]
12.5 : Any["PAKA", "HUNT", "LKRE", "EWJE"]
13.0 : Any["PAKA", "HUNT", "HAYA", "EWJE"]
13.5 : Any["OCLA", "HAYA", "TBOS", "SAWA"]
14.0 : Any["OCLA", "TBOS", "SAWA"]
14.5 : Any["HOST", "YFER"]
15.0 : Any["JOBA", "HOST", "YFER"]
15.5 : Any["JOBA", "RMAJ", "SAWA"]
16.0 : Any["IAND", "RMAJ", "DRDO", "SAWA"]
16.5 : Any["MOME", "IAND", "JKNO", "DRDO"]
17.0 : Any["MOME", "IAND", "PAKA", "JKNO"]
17.5 : Any["PAKA", "STAD", "HOST"]
18.0 : Any["STAD", "HOST", "YFER", "JKLO"]
18.5 : Any["DUNN", "YFER", "JKLO"]
19.0 : Any["STEF", "DUNN"]
19.5 : Any["STEF", "KALA", "TBOS"]
